# Data Indexing and Selection

Previously, we looked in detail at methods and tools to access, set, and modify values in NumPy arrays.
These included indexing (e.g., ``arr[2, 1]``), slicing (e.g., ``arr[:, 1:5]``), masking (e.g., ``arr[arr > 0]``), fancy indexing (e.g., ``arr[0, [1, 5]]``), and combinations thereof (e.g., ``arr[:, [1, 5]]``).
Here we'll look at similar means of accessing and modifying values in Pandas ``Series`` and ``DataFrame`` objects.
If you have used the NumPy patterns, the corresponding patterns in Pandas will feel very familiar, though there are a few quirks to be aware of.

We'll start with the simple case of the one-dimensional ``Series`` object, and then move on to the more complicated two-dimesnional ``DataFrame`` object.

In [20]:
import pandas as pd
import numpy as np

### Indexers: loc, iloc

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [3]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [4]:
# explicit index when indexing
data[1]

'a'

In [5]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [6]:
data.loc[1]

'a'

In [7]:
data.loc[1:3]

1    a
3    b
dtype: object

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

In [8]:
data.iloc[1]

'b'

In [9]:
data.iloc[1:3]

3    b
5    c
dtype: object

One guiding principle of Python code is that "explicit is better than implicit."
The explicit nature of ``loc`` and ``iloc`` make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

## Data Selection in DataFrame

Recall that a ``DataFrame`` acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of ``Series`` structures sharing the same index.
These analogies can be helpful to keep in mind as we explore data selection within this structure.

### DataFrame as a dictionary

The first analogy we will consider is the ``DataFrame`` as a dictionary of related ``Series`` objects.
Let's return to our example of areas and populations of states:

In [10]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


The individual ``Series`` that make up the columns of the ``DataFrame`` can be accessed via dictionary-style indexing of the column name:

In [11]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Equivalently, we can use attribute-style access with column names that are strings:

In [12]:
data.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

This dictionary-style syntax can also be used to modify the object, in this case adding a new column:

In [15]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


This shows a preview of the straightforward syntax of element-by-element arithmetic between ``Series`` objects.

Note that we can also just get the unterlying NumPy array representation of the ``DataFrame`` via the ``values`` attribute if we ever need to.

In [16]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

In [22]:
data["myarray"] = np.array([1, 2, 3, 4, 5])  # or assign a new column
data

,area,pop,density,array,myarray
California,423967,38332521,90.000000,1,1
Texas,695662,26448193,38.018740,2,2
New York,141297,19651127,139.076746,3,3
Florida,170312,19552860,114.806121,4,4
Illinois,149995,12882135,85.883763,5,5


### Math operations

We can also apply NumPy ufuncs on the DataFrame. The difference to evaluating the ufunc on a NumPy array is that the index is preserved.

In [29]:
np.cos(data)

,area,pop,density,array,myarray
California,-0.938208,0.319016,-0.448074,0.540302,0.540302
Texas,0.958484,-0.578606,0.949352,-0.416147,-0.416147
New York,0.745966,0.999992,0.662482,-0.989992,-0.989992
Florida,0.999781,-0.429095,-0.137551,-0.653644,-0.653644
Illinois,-0.942339,-0.972848,-0.488208,0.283662,0.283662


To get a single column, we can pass a single "index" to a ``DataFrame`` accesses a column:

In [30]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [31]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [19]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


## Eval and Query: Compound Expressions

We can achieve the same results as above using the ``eval`` and ``query`` methods.

The difference is that we write an expression in a string, which is evaluated in an optimized way. For non-large DataFrames, this is not necessary, but for large DataFrames (millions, complex expressions), it can be faster.

In [23]:
data_new = data.eval('density2 = pop * 2 / area')
data_new

,area,pop,density,array,myarray,density2
California,423967,38332521,90.000000,1,1,180.827852
Texas,695662,26448193,38.018740,2,2,76.037481
New York,141297,19651127,139.076746,3,3,278.153492
Florida,170312,19552860,114.806121,4,4,229.612241
Illinois,149995,12882135,85.883763,5,5,171.767526


In [25]:
# or inplace
data_new.eval('density3 = pop * 3 / area', inplace=True)
data_new

,area,pop,density,array,myarray,density2,density3
California,423967,38332521,90.000000,1,1,180.827852,271.241778
Texas,695662,26448193,38.018740,2,2,76.037481,114.056221
New York,141297,19651127,139.076746,3,3,278.153492,417.230238
Florida,170312,19552860,114.806121,4,4,229.612241,344.418362
Illinois,149995,12882135,85.883763,5,5,171.767526,257.651288


In [26]:
df_sel = data_new.query('density2 > 100')
df_sel

,area,pop,density,array,myarray,density2,density3
California,423967,38332521,90.000000,1,1,180.827852,271.241778
New York,141297,19651127,139.076746,3,3,278.153492,417.230238
Florida,170312,19552860,114.806121,4,4,229.612241,344.418362
Illinois,149995,12882135,85.883763,5,5,171.767526,257.651288


In [27]:
# or inplace
data_new.query('density3 < 400 & area < 400_000', inplace=True)
data_new

,area,pop,density,array,myarray,density2,density3
Florida,170312,19552860,114.806121,4,4,229.612241,344.418362
Illinois,149995,12882135,85.883763,5,5,171.767526,257.651288


## Why use eval and query?

In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

100 loops, best of 3: 3.39 ms per loop


Remember, NumPy is fast, because it pushes the loop into the compiled layer. But this abstraction can become less efficient when computing compound expressions.


For example, consider the following expression:

In [32]:
mask = (x > 0.5) & (y < 0.5)

NameError: name 'x' is not defined

Because NumPy evaluates each subexpression, this is roughly equivalent to the following:

In [4]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

In other words, *every intermediate step is explicitly allocated in memory*. If the ``x`` and ``y`` arrays are very large, this can lead to significant memory and computational overhead.
The Numexpr library gives you the ability to compute this type of compound expression element by element, without the need to allocate full intermediate arrays.
The [Numexpr documentation](https://github.com/pydata/numexpr) has more details, but for the time being it is sufficient to say that the library accepts a *string* giving the NumPy-style expression you'd like to compute:

In [5]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

The benefit here is that Numexpr evaluates the expression in a way that does not use full-sized temporary arrays, and thus can be much more efficient than NumPy, especially for large arrays.
The Pandas ``eval()`` and ``query()`` tools that we will discuss here are conceptually similar, and depend on the Numexpr package.

## ``pandas.eval()`` for Efficient Operations

The ``eval()`` function in Pandas uses string expressions to efficiently compute operations using ``DataFrame``s.
For example, consider the following ``DataFrame``s:

In [6]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

To compute the sum of all four ``DataFrame``s using the typical Pandas approach, we can just write the sum:

In [7]:
%timeit df1 + df2 + df3 + df4

10 loops, best of 3: 87.1 ms per loop


The same result can be computed via ``pd.eval`` by constructing the expression as a string:

In [8]:
%timeit pd.eval('df1 + df2 + df3 + df4')

10 loops, best of 3: 42.2 ms per loop


The ``eval()`` version of this expression is about 50% faster (and uses much less memory), while giving the same result:

In [9]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True